In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

# Загрузка датасета CIFAR10
train_data = datasets.CIFAR10(root="./cifar10_data", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.CIFAR10(root="./cifar10_data", train=False, download=True, transform=transforms.ToTensor())

# Разделим тренировочную выборку на train и val
train_size = int(len(train_data) * 0.8)
val_size = len(train_data) - train_size
train_data, val_data = torch.utils.data.random_split(train_data, [train_size, val_size])

# Заводим даталоадеры
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

# Определим модель
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Первый сверточный слой
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)        # Слой подвыборки
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # Второй сверточный слой
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # Полносвязный слой
        self.fc2 = nn.Linear(128, 10)          # Выходной слой (10 классов)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Прямой проход через первый сверточный слой
        x = self.pool(F.relu(self.conv2(x)))  # Прямой проход через второй сверточный слой
        x = x.view(-1, 64 * 8 * 8)              # Преобразуем в вектор
        x = F.relu(self.fc1(x))                # Прямой проход через первый полносвязный слой
        x = self.fc2(x)                        # Прямой проход через выходной слой
        return x

# Создаем экземпляр модели
model = SimpleCNN()

# Устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Определяем критерий потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучаем модель
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Устанавливаем модель в режим обучения
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Перемещаем данные на GPU (если доступен)

        optimizer.zero_grad()  # Обнуляем градиенты
        outputs = model(images)  # Прямой проход
        loss = criterion(outputs, labels)  # Вычисляем функцию потерь
        loss.backward()  # Обратный проход
        optimizer.step()  # Обновляем веса

    # Выводим информацию о ходе обучения
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Сохраняем модель
torch.save(model.state_dict(), 'model.pth')

# Проверка модели на тестовой выборке
model.eval()  # Устанавливаем модель в режим оценки
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Тестовая точность: {correct / total:.4f}')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.1450
Epoch [2/10], Loss: 1.0285
Epoch [3/10], Loss: 0.9789
Epoch [4/10], Loss: 0.8423
Epoch [5/10], Loss: 0.8064
Epoch [6/10], Loss: 1.0637
Epoch [7/10], Loss: 0.8102
Epoch [8/10], Loss: 0.6826
Epoch [9/10], Loss: 0.7186
Epoch [10/10], Loss: 0.7439
Тестовая точность: 0.6971


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

# Загрузка датасета CIFAR10
train_data = datasets.CIFAR10(root="./cifar10_data", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.CIFAR10(root="./cifar10_data", train=False, download=True, transform=transforms.ToTensor())

# Разделим тренировочную выборку на train и val
train_size = int(len(train_data) * 0.8)
val_size = len(train_data) - train_size
train_data, val_data = torch.utils.data.random_split(train_data, [train_size, val_size])

# Заводим даталоадеры
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

# Определим модель
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # 64 фильтра, размер 8x8 после свертки и подвыборки
        self.fc2 = nn.Linear(128, 10)  # 10 классов для CIFAR-10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Преобразуем в вектор
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Создаем экземпляр модели
model = SimpleCNN()

# Определяем критерий потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучаем модель
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Устанавливаем модель в режим обучения
    for images, labels in train_loader:
        optimizer.zero_grad()  # Обнуляем градиенты
        outputs = model(images)  # Прямой проход
        loss = criterion(outputs, labels)  # Вычисляем функцию потерь
        loss.backward()  # Обратный проход
        optimizer.step()  # Обновляем веса

    # Выводим информацию о ходе обучения
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Сохраняем модель
torch.save(model.state_dict(), 'model.pth')

# Проверка модели на тестовой выборке
model.eval()  # Устанавливаем модель в режим оценки
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Тестовая точность: {correct / total:.4f}')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.1113
Epoch [2/10], Loss: 1.1301
Epoch [3/10], Loss: 0.8959
Epoch [4/10], Loss: 1.0134
Epoch [5/10], Loss: 0.8609
Epoch [6/10], Loss: 0.6638
Epoch [7/10], Loss: 0.7375
Epoch [8/10], Loss: 0.5603
Epoch [9/10], Loss: 0.6296
Epoch [10/10], Loss: 0.6963
Тестовая точность: 0.6952
